# Latent diffusion attempt


In [1]:
!pip install einops -q
!pip install lpips -q

import os
import torch
from torch import nn
import torch.nn.functional as F
from einops import rearrange

import numpy as np

from torch.optim import Adam
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import tqdm
from tqdm.notebook import trange, tqdm
from torch.optim.lr_scheduler import MultiplicativeLR, LambdaLR

import matplotlib.pyplot as plt

from torchvision.utils import make_grid

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.5 MB/s eta 0:00:00


In [ ]:
lin1 = torch.nn.Linear(64, 32)
lin2 = torch.nn.Linear(32, 16)
x = torch.rand((16,4096, 64))
lin2(lin1(x)).shape

torch.Size([16, 4096, 16])

General diffusion utils

In [ ]:
beta_min = 1e-4
beta_max = 0.02

num_steps = 500
device = "cuda" if torch.cuda.is_available() else "cpu"

betas = np.linspace(beta_min, beta_max, num_steps)
betas = torch.from_numpy(betas).float().to(DEVICE)

alphas = 1.0 - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)

sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)
log_one_minus_alphas_cumprod = torch.log(1.0 - alphas_cumprod)
sqrt_recip_alphas_cumprod = torch.sqrt(1.0 / alphas_cumprod)
sqrt_recipm1_alphas_cumprod = torch.sqrt(1.0 / alphas_cumprod - 1)

posterior_variance = betas * (1.0 - alphas_cumprod_prev) / (1.0 - alphas_cumprod)
posterior_log_variance_clipped = torch.log(posterior_variance.clamp(min=1e-20))
posterior_mean_coef1 = betas * torch.sqrt(alphas_cumprod_prev) / (1.0 - alphas_cumprod)
posterior_mean_coef2 = (
    (1.0 - alphas_cumprod_prev) * torch.sqrt(alphas) / (1.0 - alphas_cumprod))

Loss function definition:

$ J := \mathbb{E}_{t\in (0, T), x_0 \sim p_0(x_0), \epsilon \sim \mathcal N(0,I)}\left[ \ \Vert  \epsilon - \epsilon_{\theta}(x_t, t) \Vert^2  \ \right] $

Considering $x_{residuals} = x_{HR}- up(x_{LR})$ as $x_t$

In [ ]:
def loss_fn_cond(model, x, y):
    """The loss function for training score-based generative models.

    Args:
      model: A PyTorch model instance that represents a
        time-dependent score-based model.
      x: A mini-batch of LR training data.
      y: Corresponding HR result.
    """
    # Estimate Residuals between LR-HR
    residuals = torch.subtract(x, y)

    # Sample time uniformly in 0, num_steps
    random_t = torch.randint(low=0, high=num_steps, size=(x.shape[0],), device=DEVICE)
    z = torch.randn_like(x)  # get normally distributed noise

    perturbed_x = (
        sqrt_alphas_cumprod[random_t][..., None, None, None] * residuals \
        + sqrt_one_minus_alphas_cumprod[random_t][..., None, None, None] * z
    )
    eps = model(perturbed_x, random_t)
    loss = torch.mean((z - eps) ** 2)
    return loss

# Defining Autoencoder


In [ ]:
class AEmodel(torch.nn.Module):

    def __init__(self, input_channels, channels=[8,16,32]):
        """Args:
          channels: The number of channels for feature maps of each resolution.
        """
        super().__init__()
        # Gaussian random feature embedding layer for time
        # Encoding layers where the resolution decreases
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, channels[0], 3, stride=1, bias=True),
            nn.BatchNorm2d(channels[0]),
            nn.SiLU(),
            nn.Conv2d(channels[0], channels[1], 3, stride=2, bias=True),
            nn.BatchNorm2d(channels[1]),
            nn.SiLU(),
            nn.Conv2d(channels[1], channels[2], 3, stride=1, bias=True),
            nn.BatchNorm2d(channels[2]))
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(channels[2], channels[1], 3, stride=1, bias=True),
            nn.BatchNorm2d(channels[1]),
            nn.SiLU(),
            nn.ConvTranspose2d(channels[1], channels[0], 3, stride=2, bias=True, output_padding=1),
            nn.BatchNorm2d(channels[0]),
            nn.SiLU(),
            nn.ConvTranspose2d(channels[0], input_channels, 3, stride=1, bias=True),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

### Autoencoder with time embdedding

Defining time embedding. (not actually clear why we may need a time embdedded autoencoder)

In [ ]:
class GaussianFourierProjection(nn.Module):
    """Gaussian random features for encoding time steps."""

    def __init__(self, embed_dim, scale=30.0):
        super().__init__()
        # Randomly sample weights (frequencies) during initialization.
        # These weights (frequencies) are fixed during optimization and are not trainable.
        self.W = nn.Parameter(torch.randn(embed_dim // 2) * scale, requires_grad=False)

    def forward(self, x):
        """ Args:
            x: torch.Tensor of dim 1 (e.g. torch.Tensor([12]))
        """
        x_proj = x[:, None] * self.W[None, :] * 2 * np.pi
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)


class Dense(nn.Module):
    """A fully connected layer that reshapes outputs to feature maps.
    Allow time repr to input additively from the side of a convolution layer.
    """

    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.dense = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.dense(x)[..., None, None]
        # this broadcast the 2d tensor to 4d, add the same value across space.


class AEmodel(nn.Module):

    def __init__(self, input_channels, channels=[4, 8, 16], embed_dim=256):
        """Args:
          channels: The number of channels for feature maps of each resolution.
        """
        super().__init__()
        pass

    def forward(self, x):
        raise NotImplementedError


Embedded dim: torch.Size([1, 256])


# Autoencoder Training
Decide whether to have a pretrained autoencoder for class domain (climate variable)


In [ ]:
se = SimpleEncoder(1)
x = torch.rand((4,1,32,32))
pytorch_total_params = sum(p.numel() for p in se.parameters() if p.requires_grad)
print(se(x).shape)


torch.Size([4, 32, 8, 8])


# Attention modules


### Channel Cross-Attention approach
Requires a joint latent representation including the climate data along with its high frequency representation to match H·W dimension at each downsampling stage.



In [84]:
class ResBlock(nn.Module):
    def __init__(self, nchannels, ndepth):
        super().__init__()
        self.ndepth = ndepth
        self.act = nn.SiLU()
        self.res_conv_blocks = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(nchannels, nchannels, 3, stride=1, padding="same", bias=True),
                nn.BatchNorm2d(nchannels),
                nn.SiLU(),
                nn.Conv2d(nchannels, nchannels, 3, stride=1, padding="same", bias=True),
                nn.BatchNorm2d(nchannels),
                )
            for _ in range(self.ndepth)])

    def forward(self, x):
        for conv_block in self.res_conv_blocks:
            x = self.act(conv_block(x) + x)
        return x


class ResImgEncoder(nn.Module):
    """Simple Image encoder to be used in the Q, K, V transcription"""
    def __init__(self, nchannels, ndepth=1, nheads=1, prehead="same"):
        """ Args:
            nchannels: (int) input x_feature channel.
            ndepth: (int) amount of ResBlock with skip connection.
            nheads: (int) factor to enlarge channel dimension before crossattention.
            prehead: (str) accepted ('preupsample','postupsample', 'same')
        """
        super(ResImgEncoder, self).__init__()

        self.outchannels = nchannels * nheads

        if prehead == "preupsample":
            # move first into outchannel dim via 1x1 convblock and go on with
            # ResBlocks at outchannels
            in_c, mid_c, out_c = nchannels, self.outchannels, self.outchannels
        elif prehead == "postupsample":
            # enlarge channel dimension at the end
            in_c, mid_c, out_c = nchannels, nchannels, self.outchannels
        elif prehead == "same":
            # Preserve same dimension
            in_c, mid_c, out_c = nchannels, nchannels, nchannels
        else:
            raise ValueError("prehead values accepted : ('preupsample','postupsample', 'same')")

        self.conv1pre = nn.Conv2d(in_c, mid_c, 1, stride=1, bias=True)
        self.res_conv_blocks = ResBlock(mid_c, ndepth)
        self.conv1post = nn.Conv2d(mid_c, out_c, 1, stride=1, bias=True)

    def forward(self, x):
        x = self.conv1pre(x)
        x = self.res_conv_blocks(x)
        x = self.conv1post(x)
        return x


class ChannelAttention(nn.Module):
    """Simple Cross-attention High Frequency Guided Module"""
    def __init__(self, nchannels, ndepth=1, nheads=1, prehead="same"):
        super(ChannelAttention, self).__init__()
        self.outchannels = nchannels if prehead=="same" else nchannels * nheads
        # Learnable d parameter
        self.d = torch.nn.Parameter(
                torch.abs(torch.rand((1)) + torch.randint(10//2, 10*2, (1,))
                ))
        self.d.requires_grad=True
        # Convolutional encoder to process Q, K, V
        self.Q_feature_encoder = ResImgEncoder(nchannels, ndepth, nheads, prehead)
        self.K_feature_encoder = ResImgEncoder(nchannels, ndepth, nheads, prehead)
        self.V_feature_encoder = ResImgEncoder(nchannels, ndepth, nheads, prehead)
        self.crossattn_encoder = ResImgEncoder(1, ndepth, nheads=1)
        self.conv1 = nn.Conv2d(self.outchannels, nchannels, kernel_size=1, stride=1, bias=False)

    def forward(self, x):
        """ Args:
        x: (torch.Tensor) LR image of dim (b, c, h, w)
        """
        b, _, h, w = x.shape
        c = self.outchannels
        # Generate Q,K,V via img encoder. Outshape:(batch, h*w, in_channels*nheads)
        Q = self.Q_feature_encoder(x).view(b, h*w, c)
        K = self.K_feature_encoder(x).view(b, h*w, c)
        V = self.V_feature_encoder(x).view(b, h*w, c)
        # Channel-wise product
        scoremats = torch.einsum("bjk,bjc->bkc", Q, K).view(b, 1, c, c)
        scoremats = self.crossattn_encoder(scoremats).view(b, c, c)
        attnmats  = F.softmax(scoremats / torch.sqrt(self.d), dim=-1)
        # Back into batch size
        ctx_vecs  = torch.einsum("bkc,bij->bic", attnmats, V).view(b, c, h, w)
        # 1x1 conv to convert into (batch, in_channel, h , w)
        ctx_vecs  = self.conv1(ctx_vecs)
        return  ctx_vecs + x


class TransformerBlock(nn.Module):
    def __init__(self, nchannels, nhidden=128, ndepth=1, nheads=1, prehead="same"):
        super().__init__()
        self.ca = ChannelAttention(nchannels, ndepth, nheads, prehead)
        self.norm1 = nn.InstanceNorm2d(nchannels)
        self.norm2 = nn.InstanceNorm2d(nchannels)
        self.mlp   = nn.Sequential(
            nn.Linear(nchannels, nhidden),
            nn.GELU(),
            nn.Linear(nhidden, nchannels),
        )
        #self.conv = nn.Conv2d(nchannels, nchannels, 3, stride=1, padding="same",
        #                      bias=True)

    def forward(self, x):
        b, c, h, w = x.shape
        x_norm = self.norm1(x)
        x = self.ca(x_norm) + x
        x_norm = self.norm2(x).view(b, h*w, c)
        x = self.mlp(x_norm).view(b, c, h, w) + x
        return x


class HF_Encoder(nn.Module):
    def __init__(self, nblocks, nchannels, nhidden, ndepth=1, nheads=1,
                 prehead="same"):
        super().__init__()
        self.transformer = nn.ModuleList([
            TransformerBlock(nchannels, nhidden, ndepth, nheads, prehead)
            for _ in range(nblocks)
        ])

    def forward(self, x):
        for encode_layer in self.transformer:
            x = encode_layer(x)
        return x


if __name__ == "__main__":
    import time
    t0 = time.time()
    nchannels = 64
    x = torch.rand((1, nchannels, 64, 64))
    net = HF_Encoder(nblocks=1, nchannels=nchannels, nhidden=64,
                     ndepth=4, nheads=1, prehead="same")

    print(f"Learnable params: {sum(p.numel() for p in net.parameters() if p.requires_grad)}")

    t1 = time.time()
    print(net(x).shape, t1-t0)

Learnable params: 926821
torch.Size([1, 64, 64, 64]) 0.022253990173339844


### Sequence based attention implementation

In [ ]:
class ImgEncoder(nn.Module):
    """Simple Image encoder to be used in the Q, K, V transcription"""
    def __init__(self, input_channels, channels=[8,16,32]):
        super(ImgEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, channels[0], 3, stride=1, padding="same", bias=True),
            nn.BatchNorm2d(channels[0]),
            nn.MaxPool2d(2, stride=2),
            nn.SiLU(),
            nn.Conv2d(channels[0], channels[1], 3, stride=1, padding="same", bias=True),
            nn.BatchNorm2d(channels[1]),
            nn.MaxPool2d(2, stride=2),
            nn.SiLU(),
            nn.Conv2d(channels[1], channels[2], 3, stride=1, padding="same", bias=True),
            nn.BatchNorm2d(channels[2])
            )
    def forward(self, x):
        """ outdim: [b, channels[-1], H/4, W/4] """
        x = self.encoder(x)
        return x # reduced H_{out} = H/4


class CrossAttention(nn.Module):
    """Simple Cross-attention High Frequency Guided Module"""
    def __init__(self, in_channel, feature_dim, hidden_dim=256, out_dim=256):
        super(CrossAttention, self).__init__()
        self.hidden_dim  = hidden_dim
        self.h, self.w = feature_dim

        # Learnable d parameter
        self.d = torch.nn.Parameter(
                torch.abs(torch.rand((1)) + torch.randint(10//2, 10*2, (1,))
                ))
        self.d.requires_grad=True

        # Convolutional encoder to process Q, K, V
        self.HF_encoder   = SimpleEncoder(in_channel)
        self.feat_encoder = SimpleEncoder(in_channel)
        self.linear = nn.Sequential(
            nn.Linear(self.h//4*self.w//4, hidden_dim, bias=True),
            nn.GELU,
            nn.Linear(hidden_dim, out_dim)
            )

    def forward(self, x, orig_hf):
        """ Args:
        x: (torch.Tensor) LR image of dim (b, c, h, w)
        orig_hf: (torch.Tensor) original High-Frequency map of dim (b, 1, h, w)
        """
        b, _, h, w = x.shape
        # tokens: with shape (batch, h*w, hdim)
        Q = self.convHF(orig_hf).view(b, h*w, self.hidden_dim)
        Q = self.linear(Q).view(b, )

        K = self.convK(x).view(b, h*w, self.hidden_dim)
        V = self.convV(x).view(b, h*w, self.hidden_dim)

        #scoremats = torch.einsum("BTH,BSH->BTS", Q, K)  # inner product of Q and K, a tensor
        # Channel-wise product
        scoremats = torch.einsum("bjk,bjc->bkc", Q, K)\
                    .view(b, 1, self.hidden_dim, self.hidden_dim)
        scoremats = self.convA(scoremats)\
                    .view(b, self.hidden_dim, self.hidden_dim)
        attnmats  = F.softmax(scoremats / torch.sqrt(self.d), dim=-1)
        #ctx_vecs  = torch.einsum("BTS,BSH->BTH", attnmats, V)
        # To (batch, hdim, h, w)
        ctx_vecs  = torch.einsum("bkc,bij->bic", attnmats, V)\
                    .view(b, self.hidden_dim, h, w)
        # 1k conv to (batch, in_channel, h , w)
        ctx_vecs  = self.conv1(ctx_vecs)
        return x + ctx_vecs


class TransformerBlock(nn.Module):
    """The transformer block that combines self-attn, cross-attn and feed forward neural net"""

    def __init__(self, in_channel, hidden_dim):# context_dim):
        super(TransformerBlock, self).__init__()
        self.attn_self = CrossAttention(in_channel, hidden_dim)
        #self.attn_cross = CrossAttention(hidden_dim, hidden_dim, context_dim)

        self.norm1 = nn.LayerNorm(hidden_dim)
        self.norm2 = nn.LayerNorm(hidden_dim)
        self.norm3 = nn.LayerNorm(hidden_dim)
        # implement a 2 layer MLP with K*hidden_dim hidden units, and nn.GeLU nonlinearity #######
        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, 3 * hidden_dim),
            nn.GELU(),
            nn.Linear(3 * hidden_dim, hidden_dim),
        )

    def forward(self, x, orig_hf):#, context=None):
        """ Args:
        x: (torch.Tensor) LR image of dim (b, c, h, w)
        orig_hf: (torch.Tensor) original High-Frequency map of dim (b, 1, h, w)
        """
        # Notice the + x as residue connections
        x = self.attn_self(self.norm1(x), orig_hf) + x
        # Notice the + x as residue connections
        #x = self.attn_cross(self.norm2(x), context=context) + x
        # Notice the + x as residue connections
        x = self.ffn(self.norm3(x)) + x
        return x


class SpatialTransformer(nn.Module):
    def __init__(self, in_channel, hidden_dim):
        super(SpatialTransformer, self).__init__()
        self.hf_encoder  = ImgEncoder(1, channels=[8,16,32])
        self.img_encoder = ImgEncoder(in_channel, channels=[8,16,32])

        self.transformer = TransformerBlock(in_channel, hidden_dim)

    def forward(self, x, orig_hf):# context=None):
        b, c, h, w = x.shape
        x_in = x
        # Combine the spatial dimensions and move the channel dimen to the end
        x = rearrange(x, "b c h w->b (h w) c")
        # Apply the sequence transformer
        x = self.transformer(x, orig_hf)
        # Reverse the process
        x = rearrange(x, "b (h w) c -> b c h w", h=h, w=w)
        # Residue
        return x + x_in

### Fully working Cross Attention mechanism with images

In [ ]:
class CrossAttention(nn.Module):
    """Simple Cross-attention High Frequency Guided Module"""
    def __init__(self, in_channel, hidden_dim=128):
        super(CrossAttention, self).__init__()
        self.hidden_dim  = hidden_dim
        # Learnable d parameter
        self.d = torch.nn.Parameter(
                torch.abs(torch.rand((1)) + torch.randint(10//2, 10*2, (1,))
                ))
        self.d.requires_grad=True

        # Convolutional encoder to process Q, K, V
        self.convHF = nn.Conv2d(1, hidden_dim, 3, 1, "same", bias=False)
        self.convK  = nn.Conv2d(in_channel, hidden_dim, 3, 1, "same", bias=False)
        self.convV  = nn.Conv2d(in_channel, hidden_dim, 3, 1, "same", bias=False)
        self.convA  = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding="same", bias=False)
        self.conv1  = nn.Conv2d(hidden_dim, in_channel, 1, bias=False)

    def forward(self, x, orig_hf):
        """ Args:
        x: (torch.Tensor) LR image of dim (b, c, h, w)
        orig_hf: (torch.Tensor) original High-Frequency map of dim (b, 1, h, w)
        """
        b, _, h, w = x.shape
        # tokens: with shape (batch, h*w, hdim)
        Q = self.convHF(orig_hf).view(b, h*w, self.hidden_dim)
        K = self.convK(x).view(b, h*w, self.hidden_dim)
        V = self.convV(x).view(b, h*w, self.hidden_dim)


        #scoremats = torch.einsum("BTH,BSH->BTS", Q, K)  # inner product of Q and K, a tensor
        # Channel-wise product
        scoremats = torch.einsum("bjk,bjc->bkc", Q, K)\
                    .view(b, 1, self.hidden_dim, self.hidden_dim)
        scoremats = self.convA(scoremats)\
                    .view(b, self.hidden_dim, self.hidden_dim)
        attnmats  = F.softmax(scoremats / torch.sqrt(self.d), dim=-1)
        #ctx_vecs  = torch.einsum("BTS,BSH->BTH", attnmats, V)
        # To (batch, hdim, h, w)
        ctx_vecs  = torch.einsum("bkc,bij->bic", attnmats, V)\
                    .view(b, self.hidden_dim, h, w)
        # 1k conv to (batch, in_channel, h , w)
        ctx_vecs  = self.conv1(ctx_vecs)
        return x + ctx_vecs


class TransformerBlock(nn.Module):
    """The transformer block that combines self-attn, cross-attn and feed forward neural net"""

    def __init__(self, in_channel, hidden_dim):# context_dim):
        super(TransformerBlock, self).__init__()
        self.attn_self = CrossAttention(in_channel, hidden_dim)
        #self.attn_cross = CrossAttention(hidden_dim, hidden_dim, context_dim)

        self.norm1 = nn.LayerNorm(hidden_dim)
        self.norm2 = nn.LayerNorm(hidden_dim)
        self.norm3 = nn.LayerNorm(hidden_dim)
        # implement a 2 layer MLP with K*hidden_dim hidden units, and nn.GeLU nonlinearity #######
        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, 3 * hidden_dim),
            nn.GELU(),
            nn.Linear(3 * hidden_dim, hidden_dim),
        )

    def forward(self, x, orig_hf):#, context=None):
        """ Args:
        x: (torch.Tensor) LR image of dim (b, c, h, w)
        orig_hf: (torch.Tensor) original High-Frequency map of dim (b, 1, h, w)
        """
        # Notice the + x as residue connections
        x = self.attn_self(self.norm1(x), orig_hf) + x
        # Notice the + x as residue connections
        #x = self.attn_cross(self.norm2(x), context=context) + x
        # Notice the + x as residue connections
        x = self.ffn(self.norm3(x)) + x
        return x


class SpatialTransformer(nn.Module):
    def __init__(self, in_channel, hidden_dim):
        super(SpatialTransformer, self).__init__()
        self.transformer = TransformerBlock(in_channel, hidden_dim)

    def forward(self, x, orig_hf):# context=None):
        b, c, h, w = x.shape
        x_in = x
        # Combine the spatial dimensions and move the channel dimen to the end
        x = rearrange(x, "b c h w->b (h w) c")
        # Apply the sequence transformer
        x = self.transformer(x, orig_hf)
        # Reverse the process
        x = rearrange(x, "b (h w) c -> b c h w", h=h, w=w)
        # Residue
        return x + x_in

if __name__=="__main__":
    in_channel = 1
    hidden_dim = 64
    ca = CrossAttention(in_channel, hidden_dim)
    x = torch.rand((4, in_channel, 64, 64))
    orig_hf = torch.rand((4, 1, 64, 64))

    out = ca(x,orig_hf)
    conv1 = nn.Conv2d(hidden_dim, 1, 1, bias=False)

    #tb = TransformerBlock(in_channel, hidden_dim)
    #norm = nn.LayerNorm(hidden_dim)
    #print(norm(x).shape)
    print("CrossAtt output", out.shape)
    #print("Transfb output:", tb(x, orig_hf).shape)


CrossAtt output torch.Size([4, 1, 64, 64])


### Orig implementation

In [ ]:
class CrossAttention(nn.Module):
    def __init__(
        self,
        embed_dim,
        hidden_dim,
        context_dim=None,
        num_heads=1,
    ):
        """
        Note: For simplicity reason, we just implemented 1-head attention.
        Feel free to implement multi-head attention! with fancy tensor manipulations.
        """
        super(CrossAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.context_dim = context_dim
        self.embed_dim = embed_dim
        self.query = nn.Linear(hidden_dim, embed_dim, bias=False)
        if context_dim is None:
            self.self_attn = True
            self.key = nn.Linear(hidden_dim, embed_dim, bias=False)  ###########
            self.value = nn.Linear(hidden_dim, hidden_dim, bias=False)  ############
        else:
            self.self_attn = False
            self.key = nn.Linear(context_dim, embed_dim, bias=False)  #############
            self.value = nn.Linear(context_dim, hidden_dim, bias=False)  ############

    def forward(self, tokens, context=None):
        # tokens: with shape [batch, sequence_len, hidden_dim]
        # context: with shape [batch, contex_seq_len, context_dim]
        if self.self_attn:
            Q = self.query(tokens)
            K = self.key(tokens)
            V = self.value(tokens)
        else:
            # implement Q, K, V for the Cross attention
            Q = self.query(tokens)
            K = self.key(context)
            V = self.value(context)
        # print(Q.shape, K.shape, V.shape)
        scoremats = torch.einsum(
            "BTH,BSH->BTS", Q, K
        )  # inner product of Q and K, a tensor
        attnmats = F.softmax(
            scoremats / np.sqrt(self.embed_dim), dim=-1
        )  # softmax of scoremats
        # print(scoremats.shape, attnmats.shape, )
        ctx_vecs = torch.einsum(
            "BTS,BSH->BTH", attnmats, V
        )  # weighted average value vectors by attnmats
        return ctx_vecs


class TransformerBlock(nn.Module):
    """The transformer block that combines self-attn, cross-attn and feed forward neural net"""

    def __init__(self, hidden_dim, context_dim):
        super(TransformerBlock, self).__init__()
        self.attn_self = CrossAttention(
            hidden_dim,
            hidden_dim,
        )
        self.attn_cross = CrossAttention(hidden_dim, hidden_dim, context_dim)

        self.norm1 = nn.LayerNorm(hidden_dim)
        self.norm2 = nn.LayerNorm(hidden_dim)
        self.norm3 = nn.LayerNorm(hidden_dim)
        # implement a 2 layer MLP with K*hidden_dim hidden units, and nn.GeLU nonlinearity #######
        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, 3 * hidden_dim),
            nn.GELU(),
            nn.Linear(3 * hidden_dim, hidden_dim),
        )

    def forward(self, x, context=None):
        # Notice the + x as residue connections
        x = self.attn_self(self.norm1(x)) + x
        # Notice the + x as residue connections
        x = self.attn_cross(self.norm2(x), context=context) + x
        # Notice the + x as residue connections
        x = self.ffn(self.norm3(x)) + x
        return x


class SpatialTransformer(nn.Module):
    def __init__(self, hidden_dim, context_dim):
        super(SpatialTransformer, self).__init__()
        self.transformer = TransformerBlock(hidden_dim, context_dim)

    def forward(self, x, context=None):
        b, c, h, w = x.shape
        x_in = x
        # Combine the spatial dimensions and move the channel dimen to the end
        x = rearrange(x, "b c h w->b (h w) c")
        # Apply the sequence transformer
        x = self.transformer(x, context)
        # Reverse the process
        x = rearrange(x, "b (h w) c -> b c h w", h=h, w=w)
        # Residue
        return x + x_in

# Diffusion model (Latent)


In [85]:
class Latent_UNet_Tranformer(nn.Module):
    """A time-dependent score-based model built upon U-Net architecture."""

    def __init__(self, channels=[4, 64, 128, 256], embed_dim=256, text_dim=256,
                 nblocks=1, ndepth=1, nheads=1, prehead="same"):
        """Initialize a time-dependent score-based network.

        Args:
          channels: The number of channels for feature maps of each resolution.
          embed_dim: The dimensionality of Gaussian random feature embeddings.
        """
        super().__init__()
        # Gaussian random feature embedding layer for time
        self.time_embed = nn.Sequential(
            GaussianFourierProjection(embed_dim=embed_dim),
            nn.Linear(embed_dim, embed_dim),
        )
        # Encoding layers where the resolution decreases
        self.conv1  = nn.Conv2d(channels[0], channels[1], 3, stride=1, bias=False)
        self.dense1 = Dense(embed_dim, channels[1])
        self.gnorm1 = nn.GroupNorm(4, num_channels=channels[1])
        self.conv2  = nn.Conv2d(channels[1], channels[2], 3, stride=2, bias=False)
        self.dense2 = Dense(embed_dim, channels[2])
        self.gnorm2 = nn.GroupNorm(4, num_channels=channels[2])
        self.attn2  = HF_Encoder(nblocks, channels[2], text_dim, ndepth, nheads, prehead)
        self.conv3  = nn.Conv2d(channels[2], channels[3], 3, stride=2, bias=False)
        self.dense3 = Dense(embed_dim, channels[3])
        self.gnorm3 = nn.GroupNorm(4, num_channels=channels[3])
        self.attn3  = HF_Encoder(nblocks, channels[3], text_dim, ndepth, nheads, prehead)

        self.tconv3 = nn.ConvTranspose2d(
            channels[3], channels[2], 3, stride=2, bias=False,
        )

        self.dense6  = Dense(embed_dim, channels[2])
        self.tgnorm3 = nn.GroupNorm(4, num_channels=channels[2])
        self.attn6   = HF_Encoder(nblocks, channels[2], text_dim, ndepth, nheads, prehead)
        self.tconv2  = nn.ConvTranspose2d(channels[2], channels[1], 3, stride=2, bias=False, output_padding=1
        )  # + channels[2]
        self.dense7  = Dense(embed_dim, channels[1])
        self.tgnorm2 = nn.GroupNorm(4, num_channels=channels[1])
        self.tconv1  = nn.ConvTranspose2d(channels[1], channels[0], 3, stride=1
        )  # + channels[1]

        # The swish activation function
        self.act = nn.SiLU()  # lambda x: x * torch.sigmoid(x)
        #self.cond_embed = nn.Embedding(nClass, text_dim)

    def forward(self, x, t, y=None):
        # Obtain the Gaussian random feature embedding for t
        embed = self.act(self.time_embed(t))
        y_embed = self.cond_embed(y).unsqueeze(1)
        # Encoding path
        ## Incorporate information from t
        h1 = self.conv1(x) + self.dense1(embed)
        ## Group normalization
        h1 = self.act(self.gnorm1(h1))
        h2 = self.conv2(h1) + self.dense2(embed)
        h2 = self.act(self.gnorm2(h2))
        h2 = self.attn2(h2)
        h3 = self.conv3(h2) + self.dense3(embed)
        h3 = self.act(self.gnorm3(h3))
        h3 = self.attn3(h3)

        # embdedded Residuals blocks representation? ###

        # Decoding path
        ## Skip connection from the encoding path
        h = self.tconv3(h3) + self.dense6(embed)
        h = self.act(self.tgnorm3(h))
        h = self.attn6(h)
        h = self.tconv2(h + h2)
        h += self.dense7(embed)
        h = self.act(self.tgnorm2(h))
        h = self.tconv1(h + h1)

        return h
        # Check it out and test !!!!